In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\Desktop\python\thesis-recsys


In [2]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

torch.set_printoptions(precision=2, sci_mode=False)
torch.manual_seed(0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
import pickle
with open("data/steam/data.pkl", "rb") as f:
    data = pd.read_pickle(f)
    #data = pickle.load(f)

In [5]:
user_col = 'user_id'
item_col = 'app_id'
train_set = data['train_set'][[user_col, item_col]].values.T
supervision_set = data['supervision_set'][[user_col, item_col]].values.T
valid_set = data['valid_set'][[user_col, item_col]].values.T
user_attr = data['user_attr']
item_attr = data['item_attr']

train_set = np.concatenate([train_set, supervision_set], axis=1)

In [6]:
# scaler_user = StandardScaler()
# user_attr_preprocess = scaler_user.fit_transform(user_attr)

# scaler_item = StandardScaler()
# item_attr_preprocess = np.copy(item_attr)
# item_attr_preprocess[:, 435:] = scaler_item.fit_transform(item_attr[:, 435:])

In [7]:
def temp_item_attr_rebuild(item_attr):
    new_item_attr = []
    for row in item_attr:
        tags = row[15]
        OS = list(row[0:3].nonzero()[0])
        avg_rat = row[4:10].nonzero()[0][0]
        price_original = row[12]
        new_item_attr.append([tags, OS, avg_rat, price_original])
    return np.array(new_item_attr, dtype=object)

In [8]:
item_attr2 = temp_item_attr_rebuild(item_attr)

In [9]:
item_attr2[0]

array([list([382, 345, 380, 228, 169, 12, 21, 322, 167, 157, 295, 74, 120, 356, 110, 222, 321, 72, 158, 370]),
       list([0, 1, 2]), 3, 29.99], dtype=object)

In [10]:
scheme = {
    "user_id": {
        "emb_dim": 8,
        "num_emb": user_attr.shape[0],
        "max_len": 1,
        "type": 'sparse'
    },
    "item_id": {
        "emb_dim": 8,
        "num_emb": item_attr.shape[0],
        "max_len": 1,
        "type": 'sparse'
    },
    "tags": {
        "emb_dim": 8,
        "num_emb": 425,
        "max_len": 20,
        "type": 'sparse'
    },
    "OS": {
        "emb_dim": 8,
        "num_emb": 3,
        "max_len": 3,
        "type": 'sparse'
    },
    "AvgCatRating": {
        "emb_dim": 8,
        "num_emb": 6,
        "max_len": 1,
        "type": 'sparse'
    },
    "PriceOriginal": {
        "type": 'dense'
    }
}

class SparseFeat(namedtuple('SparseFeat', ['name', 'emb_dim', 'num_emb', "max_len", "index", 'pad_index'])):
    def __new__(cls, name, emb_dim, num_emb, max_len, index, pad_index):
        return super(SparseFeat, cls).__new__(cls, name, emb_dim, num_emb, max_len, index, pad_index)
    
class DenseFeat(namedtuple('DenseFeat', ['name', "index", 'pad_index'])):
    def __new__(cls, name, index, pad_index):
        return super(DenseFeat, cls).__new__(cls, name, index, pad_index)    
    
# class VarLenSparseFeat(namedtuple('VarLenSparseFeat', ['name', 'emb_dim', 'num_emb', "max_len", 'index'])):
#     def __new__(cls, name, emb_dim, num_emb, max_len, index):
#         return super(VarLenSparseFeat, cls).__new__(cls, name, emb_dim, num_emb, max_len, index)

# TODO: start+=feat['max_len'] powoduje blad, do sprawdzenia
class FeatureStore:
    def __init__(self, scheme):
        self.features = {'sparse': [], 'dense': []}
        self.n_features = len(scheme.keys())
        
        start = 0
        for i, (feat_name, feat) in enumerate(scheme.items()):
            if feat['type'] == 'sparse':
                self.features['sparse'].append(
                    SparseFeat(feat_name, feat['emb_dim'], feat['num_emb'], feat['max_len'], i, (start, start+feat['max_len']))
                )
                start = start + feat['max_len']
            elif feat['type'] == 'dense':
                self.features['dense'].append(
                    DenseFeat(feat_name, i, (start, start+1))
                )
                start += 1

        self.sparse_index = self.get_feature_index('sparse')
        self.dense_index = self.get_feature_index('dense')

    def input_len(self):
        sparse_len = sum([f.emb_dim for f in self.features['sparse']])
        dense_len = len(self.features['dense'])
        return sparse_len + dense_len
    
    def emb_dim(self):
        return 8
    
    def num_emb(self):
        return sum([f.num_emb for f in self.features['sparse']])

    def get_feature_index(self, feat_type):
        return torch.hstack([torch.arange(i[0], i[1]) for i in [f.pad_index for f in self.features[feat_type]]])

In [11]:
feature_store = FeatureStore(scheme)

In [12]:
class DeepFMDataset(Dataset):
    def __init__(self, edge_index, user_attr, item_attr):
        self.edge_index = edge_index
        self.user_attr = user_attr
        self.item_attr = item_attr
        
        self.users = edge_index[0]
        self.items = edge_index[1]
    
    def __len__(self):
        return self.edge_index.shape[1]
    
    def __getitem__(self, idx):
        u_id = self.users[idx]
        i_id = self.items[idx]
        
        #u_attr = self.user_attr[u_id]
        u_attr = []
        i_attr = self.item_attr[i_id]

        x = np.concatenate([[u_id], [i_id], u_attr, i_attr])
        
        sparse_features = self._get_sparse_features(x)
        dense_features = self._get_dense_features(x)
        
        return sparse_features, dense_features
    
    def _get_sparse_features(self, x):
        indices = [f.index for f in feature_store.features['sparse']]
        return x[indices]
    
    def _get_dense_features(self, x):
        indices = [f.index for f in feature_store.features['dense']]
        return x[indices]

def pad_sequence_max_len(sequence, max_len):
    sequence[0] = pad(sequence[0], (0, max_len-sequence[0].shape[0]), value=-1)
    return pad_sequence(sequence, batch_first=True, padding_value=-1) + 1

def collate_fn(batch):
    xs = [[] for f in range(feature_store.n_features)]
    for (_sparse, _dense) in batch:
        for i, f in enumerate(feature_store.features['sparse']):
            xs[f.index].append(torch.tensor(_sparse[i], dtype=torch.float32))

        for i, f in enumerate(feature_store.features['dense']):
            xs[f.index].append(torch.tensor(_dense[i], dtype=torch.float32))

    for i, f in enumerate(feature_store.features['sparse']):
        if f.max_len > 1:
            xs[f.index] = pad_sequence_max_len(xs[f.index], f.max_len)
        else:
            xs[f.index] = torch.tensor(xs[f.index]) + 1
        
    for i, f in enumerate(feature_store.features['dense']):
        xs[f.index] = torch.tensor(xs[f.index])
    
    xs = torch.column_stack(xs)

    return xs.to(device)

In [13]:
dataset = DeepFMDataset(train_set, user_attr, item_attr2)
dataloader = DataLoader(dataset, shuffle=False, batch_size=1024, collate_fn=collate_fn)

In [14]:
batch = next(iter(dataloader))

In [15]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, emb_size):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.u = nn.Embedding(n_users, emb_size)
        self.i = nn.Embedding(n_items, emb_size)
        
    def forward(self, ux, ix):
        return torch.sum(self.u(ux) * self.i(ix), dim=1)
        #return torch.sum((self.u(ux) * self.i(ix)).squeeze(1), dim=1)

In [16]:
feature_store.features['sparse']

[SparseFeat(name='user_id', emb_dim=8, num_emb=3066721, max_len=1, index=0, pad_index=(0, 1)),
 SparseFeat(name='item_id', emb_dim=8, num_emb=1231, max_len=1, index=1, pad_index=(1, 2)),
 SparseFeat(name='tags', emb_dim=8, num_emb=425, max_len=20, index=2, pad_index=(2, 22)),
 SparseFeat(name='OS', emb_dim=8, num_emb=3, max_len=3, index=3, pad_index=(22, 25)),
 SparseFeat(name='AvgCatRating', emb_dim=8, num_emb=6, max_len=1, index=4, pad_index=(25, 26))]

In [17]:
# class EmbeddingBagCollection:
#     """
#     EmbeddingBagCollection represents a collection of pooled embeddings (`EmbeddingBags`).

#     It processes sparse data in the form of `KeyedJaggedTensor` with values of the form
#     [F X B X L] where:

#     * F: features (keys)
#     * B: batch size
#     * L: length of sparse features (jagged)

#     and outputs a `KeyedTensor` with values of the form [B * (F * D)] where:

#     * F: features (keys)
#     * D: each feature's (key's) embedding dimension
#     * B: batch size

#     Args:
#         tables (List[EmbeddingBagConfig]): list of embedding tables.
#         is_weighted (bool): whether input `KeyedJaggedTensor` is weighted.
#         device (Optional[torch.device]): default compute device.

#     Example::

#         table_0 = EmbeddingBagConfig(
#             name="t1", embedding_dim=3, num_embeddings=10, feature_names=["f1"]
#         )
#         table_1 = EmbeddingBagConfig(
#             name="t2", embedding_dim=4, num_embeddings=10, feature_names=["f2"]
#         )

#         ebc = EmbeddingBagCollection(tables=[table_0, table_1])

#         #        0       1        2  <-- batch
#         # "f1"   [0,1] None    [2]
#         # "f2"   [3]    [4]    [5,6,7]
#         #  ^
#         # feature

#         features = KeyedJaggedTensor(
#             keys=["f1", "f2"],
#             values=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7]),
#             offsets=torch.tensor([0, 2, 2, 3, 4, 5, 8]),
#         )

#         pooled_embeddings = ebc(features)
#         print(pooled_embeddings.values())
#         tensor([[-0.8899, -0.1342, -1.9060, -0.0905, -0.2814, -0.9369, -0.7783],
#             [ 0.0000,  0.0000,  0.0000,  0.1598,  0.0695,  1.3265, -0.1011],
#             [-0.4256, -1.1846, -2.1648, -1.0893,  0.3590, -1.9784, -0.7681]],
#             grad_fn=<CatBackward0>)
#         print(pooled_embeddings.keys())
#         ['f1', 'f2']
#         print(pooled_embeddings.offset_per_key())
#         tensor([0, 3, 7])
#     """

#     def __init__(self, feature_store: FeatureStore) -> None:
#         super().__init__()
#         self.embedding_bags: nn.ModuleDict = nn.ModuleDict()
#         self._feature_store = feature_store

#         for feature_config in feature_store.features['sparse']:
#             self.embedding_bags[feature_config.name] = nn.Embedding(
#                 num_embeddings=feature_config.num_emb,
#                 embedding_dim=embedding_config.emb_dim,
#                 padding_idx=-1,
#                 dtype=torch.float32,
#                 device=device,
#             )


#     def forward(self, x):
#         """
#         Args:
#             features (KeyedJaggedTensor): KJT of form [F X B X L].

#         Returns:
#             KeyedTensor
#         """

#         pooled_embeddings: List[torch.Tensor] = []

#         feature_dict = features.to_dict()
#         for i, embedding_bag in enumerate(self.embedding_bags.values()):
#             for feature_name in self._feature_names[i]:
#                 f = feature_dict[feature_name]
#                 res = embedding_bag(
#                     input=f.values(),
#                     offsets=f.offsets(),
#                     per_sample_weights=f.weights() if self._is_weighted else None,
#                 ).float()
#                 pooled_embeddings.append(res)
#         data = torch.cat(pooled_embeddings, dim=1)
#         return KeyedTensor(
#             keys=self._embedding_names,
#             values=data,
#             length_per_key=self._lengths_per_embedding,
#         )

In [18]:
class CrossProductNet(nn.Module):
    def forward(self, emb_x):
        cross = emb_x @ emb_x.permute(1, 0)
        triangle_lower_cross = torch.tril(cross, diagonal=-1)
        return torch.sum(triangle_lower_cross)

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.1, batch_norm=True):
        super(MLP, self).__init__()
        
        _layers = []
        dims = [input_dim] + hidden_dim + [1]
        for in_dim, out_dim in zip(dims, dims[1:]):
            _layers.append(nn.Linear(in_dim, out_dim))
            if batch_norm:
                _layers.append(nn.BatchNorm1d(out_dim))
            _layers.append(nn.ReLU())
            _layers.append(nn.Dropout(p=dropout))

        self.layers = nn.ModuleList(_layers)
        
    def forward(self, emb_x):
        return self.layers(emb_x)
    
class EmbeddingNet(nn.Module):
    def __init__(self, feature_store):
        super(EmbeddingNet, self).__init__()
        _embeddings = {feat.name: nn.Embedding(feat.num_emb, feat.emb_dim, padding_idx=0, device=device) for feat in feature_store.features['sparse']}
        self._feature_store = feature_store
        self.embeddings = nn.ModuleDict(_embeddings)
        
    def forward(self, x):
        x_emb = []
        for name, embed_matrix in self.embeddings.items():
            feat = [i for i in self._feature_store.features['sparse'] if i.name == name][0]
            emb = embed_matrix(x[feat.pad_index[0]:feat.pad_index[1]].to(torch.long))
            print(f"FEAT: {feat} \nX: {x[feat.pad_index[0]:feat.pad_index[1]].to(torch.long)}\n X_EMB: {emb}\n")
            emb_agg = torch.sum(emb, axis=0)
            print(emb_agg)
            x_emb.append(emb_agg)
        x_emb = torch.cat(x_emb, axis=1)
        return emb   

class DeepFM(nn.Module):
    def __init__(self, feature_store, hidden_dim):
        super(DeepFM, self).__init__()
        self.feature_store = feature_store

        self.V = EmbeddingNet(feature_store)
        self.fm = CrossProductNet()
        self.dnn = MLP(input_dim, hidden_dim)
        
    def forward(self, x):
        x_dense = x[feature_store.dense_index]
        
        x_sparse = self.V(x)
        x_sparse_dense = torch.cat([x_sparse, x_dense])
        x = self.fm(x_sparse_emb) + self.dnn(x_sparse_dense)
        return x

In [19]:
en = EmbeddingNet(feature_store)

In [24]:
en.embeddings

ModuleDict(
  (user_id): Embedding(3066721, 8, padding_idx=0)
  (item_id): Embedding(1231, 8, padding_idx=0)
  (tags): Embedding(425, 8, padding_idx=0)
  (OS): Embedding(3, 8, padding_idx=0)
  (AvgCatRating): Embedding(6, 8, padding_idx=0)
)

In [20]:
feature_store.features

{'sparse': [SparseFeat(name='user_id', emb_dim=8, num_emb=3066721, max_len=1, index=0, pad_index=(0, 1)),
  SparseFeat(name='item_id', emb_dim=8, num_emb=1231, max_len=1, index=1, pad_index=(1, 2)),
  SparseFeat(name='tags', emb_dim=8, num_emb=425, max_len=20, index=2, pad_index=(2, 22)),
  SparseFeat(name='OS', emb_dim=8, num_emb=3, max_len=3, index=3, pad_index=(22, 25)),
  SparseFeat(name='AvgCatRating', emb_dim=8, num_emb=6, max_len=1, index=4, pad_index=(25, 26))],
 'dense': [DenseFeat(name='PriceOriginal', index=5, pad_index=(26, 27))]}

In [21]:
batch[1]

tensor([  2.00,   2.00,  17.00, 162.00,  65.00,  23.00, 159.00, 348.00, 340.00,
        323.00, 362.00,  47.00,  88.00, 109.00, 105.00, 134.00,  37.00, 345.00,
        285.00,   0.00,   0.00,   0.00,   1.00,   0.00,   0.00,   6.00,  19.99],
       device='cuda:0')

In [22]:
en(batch[1])

FEAT: SparseFeat(name='user_id', emb_dim=8, num_emb=3066721, max_len=1, index=0, pad_index=(0, 1)) 
X: tensor([2], device='cuda:0')
 X_EMB: tensor([[-0.13,  1.22,  1.44,  1.06, -0.49, -1.42, -0.72, -1.30]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)

tensor([-0.13,  1.22,  1.44,  1.06, -0.49, -1.42, -0.72, -1.30],
       device='cuda:0', grad_fn=<SumBackward1>)
FEAT: SparseFeat(name='item_id', emb_dim=8, num_emb=1231, max_len=1, index=1, pad_index=(1, 2)) 
X: tensor([2], device='cuda:0')
 X_EMB: tensor([[-0.50,  0.59,  1.34,  1.16, -0.18,  0.08,  0.91, -1.04]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)

tensor([-0.50,  0.59,  1.34,  1.16, -0.18,  0.08,  0.91, -1.04],
       device='cuda:0', grad_fn=<SumBackward1>)
FEAT: SparseFeat(name='tags', emb_dim=8, num_emb=425, max_len=20, index=2, pad_index=(2, 22)) 
X: tensor([ 17, 162,  65,  23, 159, 348, 340, 323, 362,  47,  88, 109, 105, 134,
         37, 345, 285,   0,   0,   0], device='cuda:0')
 X_EMB: tensor([[-0.9

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
model = DeepFM(feature_store, hidden_dim=[128,64])

In [ ]:
torch.tensor([[1,2,3], [4,5,6], [7,8,9]])[[1,2]]

In [ ]:
model

In [ ]:
features.features

In [ ]:
train_dataset[9432]

In [ ]:
filter_input_types(train_dataset[9432], features)

In [ ]:
e = EmbeddingNet([f.num_emb for f in features], 8)

In [ ]:
np.array((0, *np.cumsum([f.num_emb for f in features])[:-1]))

In [ ]:
[f.num_emb for f in features]

In [ ]:
e.offsets

In [ ]:
torch.arange(20).new_tensor([0, 5, 10]).unsqueeze(0)

In [ ]:
x_input = {
    "user_id": 5,
    "item_id": 20,
    "tags": [0, 5, 8, 100],
    "OS": [0, 1],
    "AvgCatRating": 3 
}
x_input_tensor = torch.tensor([5, 
                               20, 
                               0, 5, 8, 100,
                               0, 1,
                               3
                              ])

In [ ]:
e(x_input)

In [ ]:
def approx_negative_sampling(u_id, i_id, n_samples, bs):
    ind = torch.arange(bs).repeat(n_samples)
    perm =  torch.randperm(ind.shape[0])
    random_indices = ind[perm]
    i_id_neg = i_id[random_indices]
    
    u_id = u_id.repeat(n_samples+1)
    i_id = torch.cat([i_id, i_id_neg])
    y_true = torch.cat([torch.ones(bs).to(device),
                       torch.zeros(bs*n_samples).to(device)])
    
    return u_id, i_id, y_true

In [ ]:
def train(n_epochs, print_loss=500):
    model.train()
    batch_size = train_loader.batch_size
    
    for epoch in range(n_epochs):
        running_loss = 0.
        preds, ground_truths = [], []
        batch_print_loss = 0.
        for i_batch, batch in enumerate(tqdm(train_loader)):
            batch = [i.to(device) for i in batch]
            u_id, i_id, u_attr, i_attr = batch
 
            u_id, i_id, y_true = approx_negative_sampling(u_id, i_id, n_samples=2, bs=batch_size)
            
            y_pred = model(u_id, i_id)
            loss = criterion(y_pred, y_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            preds.append(y_pred)
            ground_truths.append(y_true)
            running_loss += loss.item()
            
            if not ((i_batch+1) % print_loss):
                pred = torch.cat(preds, dim=0).detach().sigmoid().cpu().numpy()
                ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
                last_loss = running_loss / print_loss
                
                train_roc_auc = roc_auc_score(ground_truth, pred)
                test_loss, test_roc_auc = test()
                
                preds, ground_truths = [], []
                running_loss = 0.
                
                print(f"batch <{i_batch}>\ntrain_loss: {last_loss} - train_roc_auc: {train_roc_auc}\ntest_loss: {test_loss} - test_roc_auc: {test_roc_auc}\n")
        print(f"Epoch: {epoch}, Loss: {running_loss / len(train_loader):.4f}")

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    batch_size = val_loader.batch_size
    
    running_loss = 0.
    preds, ground_truths = [], []

    for i_batch, batch in enumerate(tqdm(val_loader)):
        batch = [i.to(device) for i in batch]
        u_id, i_id, u_attr, i_attr = batch
        
        u_id, i_id, y_true = approx_negative_sampling(u_id, i_id, n_samples=2, bs=batch_size)

        y_pred = model(u_id, i_id)
        loss = criterion(y_pred, y_true)
        
        preds.append(y_pred)
        ground_truths.append(y_true)
        running_loss += loss.item()
        
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    
    test_loss = running_loss / len(val_loader)
    test_score = roc_auc_score(ground_truth, pred)

    return test_loss, test_score

In [ ]:
train_dataset = MFDataset(train_set, data['user_attr'], data['item_attr'])
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, drop_last=True)

val_dataset = MFDataset(valid_set, data['user_attr'], data['item_attr'])
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, drop_last=True)

In [ ]:
n_users, n_items = user_attr.shape[0], item_attr.shape[0]

model = MF(n_users, n_items, emb_size=32)
model = model.to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
train(n_epochs=10, print_loss=1000)

In [ ]:
model

In [ ]:
test()

In [ ]:
with torch.no_grad():
    batch_size = train_loader.batch_size
    batch = next(iter(train_loader))
    batch = [i.to(device) for i in batch]
    u_id, i_id, u_attr, i_attr = batch
    
    neg_item_idx = approx_negative_sampling(i_id, bs=batch_size)

    u_id = torch.cat([u_id, u_id])
    i_id = torch.cat([i_id, neg_item_idx])
    y_true = torch.cat([torch.ones(batch_size).to(device),
                        torch.zeros(batch_size).to(device)])

    y_pred = model(u_id, i_id)
    
    print(y_pred.sigmoid(), y_true)

In [ ]:
criterion(y_pred, y_true)

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(path):
    model = MF(n_users, n_items, emb_size=32)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [ ]:
#save_model(model, "models/mf_01.pth")

In [ ]:
model = load_model("models/mf_01.pth")

In [ ]:
from reco_env import RecoEnv
from utils import import_data_for_env
import gym

In [ ]:
env = gym.make(RecoEnv.id, **import_data_for_env())

In [ ]:
vc = rec.user_id.value_counts()

In [ ]:
vc

In [ ]:
vc[vc >= 3]

In [ ]:
plt.plot(vc)